In [1]:
############################### DEPENDENCIES ##################################

import json
import os
import sys
import contextlib
import io
import numpy as np
import inspect
import os.path
import matplotlib.pyplot as plt
from math import pi, atan, sin, sqrt, tanh, cosh, exp, ceil
import seaborn as sns
from numpy.core.fromnumeric import shape
import random
import time
import math
import statistics
from pathlib import Path

In [11]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

print(module_path)

project_root = module_path

c:\Users\aurel\Documents\GitHub\Code_Thesis_GitHub\Code_Thesis_GitHub


In [6]:
#filename = inspect.getframeinfo(inspect.currentframe()).filename
#file_path = os.path.dirname(os.path.abspath(filename))
#project_root = os.path.dirname(file_path)

#if project_root not in sys.path:
#    sys.path.append(project_root)

#print(project_root)

C:\Users\aurel\AppData\Local\Temp


In [12]:
from event_model import event_model as em
from validator import validator_lite as vl
import data_analysis.event_generator as eg
from visual.color_map import Colormap

In [295]:
def load_event(file_name, plot_event=False):
# NB: some modifications might have been done for a better understanding
    f = open(file_name)
    json_data_event = json.loads(f.read())

    ev = em.event(json_data_event, read_tracks=True)

    modules = ev.modules
    tracks = ev.real_tracks

    if plot_event:
        eg.plot_tracks_and_modules(tracks, modules, title="Loaded Event")

    modules_even = []
    modules_odd = []

    for i in range(len(modules)):
        if i % 2 == 0:
            modules_even.append(modules[i])
        else:
            modules_odd.append(modules[i])

    return json_data_event, modules_even, modules_odd, tracks, modules

In [296]:
i = 1
event_file_name = "/datasets/minibias/velo_event_"
file_name = project_root + event_file_name
print(file_name)

c:\Users\aurel\Documents\GitHub\Code_Thesis_GitHub\Code_Thesis_GitHub/datasets/minibias/velo_event_


In [297]:
json_data_event, modules_even, modules_odd, tracks, modules = load_event(file_name + str(i) + ".json", plot_event=False)

> Idea behind the code: For each event, for each trace, count the number of hits in total and then the number of hits in each "type" of modules


In [311]:
# For 1 event
number_hits_by_track = []
 
counter_by_track_odd = []
counter_by_track_even =[]

number_of_tracks = len(tracks)
print(f'Total number of tracks: {number_of_tracks}')

for track in tracks:
    number_hits = len(track.hits)
    number_hits_by_track.append(number_hits)

for track in tracks:
    counter_hits_odd_module = 0
    counter_hits_even_module = 0
    for hits in track:
        if hits.module_number % 2 == 1:
            counter_hits_odd_module +=1
        else:
            counter_hits_even_module +=1
    counter_by_track_odd.append(counter_hits_odd_module)
    counter_by_track_even.append(counter_hits_even_module)

#print(number_hits_by_track)
#print(counter_by_track_odd)
#print(counter_by_track_even)

percentage_of_tracks_odd_modules = [x/y for x,y in zip(counter_by_track_odd,number_hits_by_track)]
percentage_of_tracks_even_modules = [x/y for x,y in zip(counter_by_track_even,number_hits_by_track)]

#print(f'Percentage in odd modules by module: {percentage_of_tracks_odd_modules}')
#print(f'Percentage in even modules by module: {percentage_of_tracks_even_modules}')

number_of_not_mixed_tracks = len([x for x in percentage_of_tracks_even_modules if x in [0,1]])
percentage_of_not_mixed_tracks = number_of_not_mixed_tracks/number_of_tracks

print(f'Total number of not mixed tracks: {number_of_not_mixed_tracks}')
print(f'Percentage of mixed tracks: {percentage_of_not_mixed_tracks}')

number_of_mixed_tracks = len([x for x in percentage_of_tracks_even_modules if not x in [0,1]])
percentage_of_mixed_tracks = number_of_mixed_tracks/number_of_tracks

print(f'Total number of mixed tracks: {number_of_mixed_tracks}')
print(f'Percentage of mixed tracks: {percentage_of_mixed_tracks}')

Total number of tracks: 344
Total number of not mixed tracks: 290
Percentage of mixed tracks: 0.8430232558139535
Total number of mixed tracks: 54
Percentage of mixed tracks: 0.1569767441860465


In [340]:
# Create the fct for 1 event
def percentage_tracks_even_odd_modules(tracks):
    number_hits_by_track = []
    counter_by_track_odd = []
    counter_by_track_even =[]

    number_of_tracks = len(tracks)

    for track in tracks:
        number_hits = len(track.hits)
        number_hits_by_track.append(number_hits)

    for track in tracks:
        counter_hits_odd_module = 0
        counter_hits_even_module = 0
        for hits in track:
            if hits.module_number % 2 == 1:
                counter_hits_odd_module +=1
            else:
                counter_hits_even_module +=1
        counter_by_track_odd.append(counter_hits_odd_module)
        counter_by_track_even.append(counter_hits_even_module)

    percentage_of_tracks_odd_modules = [x/y for x,y in zip(counter_by_track_odd,number_hits_by_track)]
    percentage_of_tracks_even_modules = [x/y for x,y in zip(counter_by_track_even,number_hits_by_track)]

    number_of_not_mixed_tracks = len([x for x in percentage_of_tracks_even_modules if x in [0,1]])
    percentage_of_not_mixed_tracks = number_of_not_mixed_tracks/number_of_tracks

    number_of_mixed_tracks = len([x for x in percentage_of_tracks_even_modules if not x in [0,1]])
    percentage_of_mixed_tracks = number_of_mixed_tracks/number_of_tracks

    return  percentage_of_not_mixed_tracks, percentage_of_mixed_tracks


In [341]:
percentage_of_not_mixed_tracks, percentage_of_mixed_tracks = percentage_tracks_even_odd_modules(tracks)
print(f'The percentage of not mixed track in the event {i} is {percentage_of_not_mixed_tracks} %.')
print(f'The percentage of mixed track in the event {i} is {percentage_of_mixed_tracks} %.')

The percentage of not mixed track in the event 999 is 0.8430232558139535 %.
The percentage of mixed track in the event 999 is 0.1569767441860465 %.


## Implementations on several events

In [303]:
# resource switch
dataset={
    'small':'datasets/small_dataset',
    'bsphiphi': 'datasets/bsphiphi',
    'minibias': 'datasets/minibias'
}


1. Bsphiphi dataset

In [360]:
DATASET = 'bsphiphi'
load_percent = 1 #between 0 to 1
shuffle = True

In [361]:
all_events = []
all_modules = []
all_tracks = []

for (dirpath, dirnames, filenames) in os.walk(os.path.abspath(os.path.join('..', dataset[DATASET]))):
    if shuffle:
        random.shuffle(filenames)

    for i, filename in enumerate(filenames):
        # Get an event
        f = open(os.path.realpath(os.path.join(dirpath, filename)))
        json_data_event = json.loads(f.read())

        ###
        ev = em.event(json_data_event, read_tracks=True)
        md = ev.modules
        tk = ev.real_tracks
        
        ###
        all_events.append(ev) 
        all_modules.append(md)
        all_tracks.append(tk)

        f.close()
        
        if i == int((load_percent * len(filenames))-1):
            break 
        
print(f'Successfully loaded {len(all_events)} events from {DATASET} dataset.' )
        

Successfully loaded 1000 events from bsphiphi dataset.


In [342]:
total_mixed_track = []
total_not_mixed_track = []
total_number_tracks  = 0

for event in all_tracks:
    for track in all_tracks:
        total_number_tracks +=1
        percentage_of_not_mixed_tracks, percentage_of_mixed_tracks = percentage_tracks_even_odd_modules(track)
        total_not_mixed_track.append(percentage_of_not_mixed_tracks)
        total_mixed_track.append(percentage_of_mixed_tracks)

    #print(f'The percentage of not mixed track in the event {i} is {percentage_of_not_mixed_tracks} %.')
    #print(f'The percentage of mixed track in the event {i} is {percentage_of_mixed_tracks} %.')
total_percentage_not_mixed_track = (sum(total_not_mixed_track)/total_number_tracks)*100
total_percentage_mixed_track = (sum(total_mixed_track)/total_number_tracks)*100

print(f'Total number of analysed tracks: {total_number_tracks} tracks.')
print(f'Total not mixed tracks: {total_percentage_not_mixed_track} %.')
print(f'Total mixed tracks: {total_percentage_mixed_track} %.')


Total number of analysed tracks: 1000000 tracks.
Total not mixed tracks: 85.79324860207895 %.
Total mixed tracks: 14.206751397870743 %.


In [362]:
def all_events_percentage_tracks_even_odd_modules(all_tracks):
    total_mixed_track = []
    total_not_mixed_track = []
    total_number_tracks  = 0

    for event in all_tracks:
        for track in all_tracks:
            total_number_tracks +=1
            percentage_of_not_mixed_tracks, percentage_of_mixed_tracks = percentage_tracks_even_odd_modules(track)
            total_not_mixed_track.append(percentage_of_not_mixed_tracks)
            total_mixed_track.append(percentage_of_mixed_tracks)

    total_percentage_not_mixed_track = (sum(total_not_mixed_track)/total_number_tracks)*100
    total_percentage_mixed_track = (sum(total_mixed_track)/total_number_tracks)*100

    print(f'Total number of analysed tracks: {total_number_tracks} tracks.')
    print(f'Total not mixed tracks: {total_percentage_not_mixed_track} %.')
    print(f'Total mixed tracks: {total_percentage_mixed_track} %.')

In [363]:
all_events_percentage_tracks_even_odd_modules(all_tracks)

Total number of analysed tracks: 1000000 tracks.
Total not mixed tracks: 85.7932486020791 %.
Total mixed tracks: 14.20675139787071 %.


2. Minibias datasets

In [364]:
DATASET = 'minibias'
load_percent = 1 #between 0 to 1
shuffle = True 

In [365]:
all_events = []
all_modules = []
all_tracks = []

for (dirpath, dirnames, filenames) in os.walk(os.path.abspath(os.path.join('..', dataset[DATASET]))):
    if shuffle:
        random.shuffle(filenames)

    for i, filename in enumerate(filenames):
        # Get an event
        f = open(os.path.realpath(os.path.join(dirpath, filename)))
        json_data_event = json.loads(f.read())

        ###
        ev = em.event(json_data_event, read_tracks=True)
        md = ev.modules
        tk = ev.real_tracks
        
        ###
        all_events.append(ev) 
        all_modules.append(md)
        all_tracks.append(tk)

        f.close()
        
        if i == int((load_percent * len(filenames))-1):
            break 
        
print(f'Successfully loaded {len(all_events)} events from {DATASET} dataset.' )
        

Successfully loaded 995 events from minibias dataset.


In [366]:
total_mixed_track = []
total_not_mixed_track = []
total_number_tracks  = 0

for event in all_tracks:
    for track in all_tracks:
        total_number_tracks +=1
        percentage_of_not_mixed_tracks, percentage_of_mixed_tracks = percentage_tracks_even_odd_modules(track)
        total_not_mixed_track.append(percentage_of_not_mixed_tracks)
        total_mixed_track.append(percentage_of_mixed_tracks)

total_percentage_not_mixed_track = (sum(total_not_mixed_track)/total_number_tracks)*100
total_percentage_mixed_track = (sum(total_mixed_track)/total_number_tracks)*100

print(f'Total number of analysed tracks: {total_number_tracks} tracks.')
print(f'Total not mixed tracks: {total_percentage_not_mixed_track} %.')
print(f'Total mixed tracks: {total_percentage_mixed_track} %.')

Total number of analysed tracks: 990025 tracks.
Total not mixed tracks: 86.00538275005681 %.
Total mixed tracks: 13.994617249947709 %.
